<img src="sentimentpost.png">,

## Applying Sentiment analysis to movie review using Kaggle labeled dataset.

https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data

### Set up Kaggle dataset to access from a Postgres DB


## Part 1- Write data to Postgres DB   
__Objective__:  

The purpose of this post besides creating and measuring the accuracy of a sentiment model to predict movie reviews is to practice most of the processes necessary to implement a working Docker environment.  From what I witnessed, many of the cool data mining and analytics I have uncovered are most useful when the results are easily accessed in a database.  This example uses Postgres database in a contained Docker-Compose environment.  Most of the essential parts of extracting, transforming, and loading (ETL) are acccomplished in this post.  
This sentiment model example uses the kaggle sample att he link above. My original sources for learning the basics of NLP included using the concepts learned in _Python 3 Text Processing with NLTK 3 Cookbook_ and _pythonprogramming.net_. In practice, I have used the concepts learned here to implement a live sentiment system for customer and support calls stored in salesforce.com. Although part 1 could have been done using Pandas, I prefer to use Postges in a Docker service to make the data persistent and allow for easily querying the data using Adminer.   


1.  import csv files downloaded from kaggle
2.  load data to a Postgres db in docker container  
3.  clean train and test tables to adapt Kaggle dataset to reflect more likely original data
---

In [ ]:
#!/usr/local/bin/python3.6
#import logging
import os
import pyodbc
import psycopg2
import psycopg2.extras
import numpy as np
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine

In [ ]:
#postgres authentication
user = "alexp"
password = "secret"
host = "pg_db"
port = "5432"
database = "priv_workspace"

### 1.  import csv files downloaded from kaggle

In [ ]:
#view file names from kaggle unzipped downloads contents:
!ls

In [ ]:
#first 10 lines
!head -2 train.tsv

In [ ]:
!head -2 test.tsv

In [ ]:
# Create a cleaned up CSV versions of files with consistent row lengths.
# Postgres COPY doesn't handle "ragged" files very well

import csv

txt_file = 'train.tsv'
csv_file =  'train.csv'

with open(txt_file, 'r') as txt_file:
    with open(csv_file, 'w') as csv_file:
        csv_writer = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
        for line in txt_file:
            if not all(ord(c) < 128 for c in line):
                line = unidecode.unidecode(line)
            row = line.rstrip('\t\r\n').split('\t')
            if len(row) != 4:
                print('skipping bad row (length %s, expected 4):' % len(row))
                print(row)
                continue
            csv_writer.writerow(row)

            
txt_file = 'test.tsv'
csv_file =  'test.csv'

with open(txt_file, 'r') as txt_file:
    with open(csv_file, 'w') as csv_file:
        csv_writer = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
        for line in txt_file:
            if not all(ord(c) < 128 for c in line):
                line = unidecode.unidecode(line)
            row = line.rstrip('\t\r\n').split('\t')
            if len(row) != 3:
                print('skipping bad row (length %s, expected 4):' % len(row))
                print(row)
                continue
            csv_writer.writerow(row)


In [ ]:
# columns names from created csv files
in_file_nm = 'train.csv'
in_file = open(in_file_nm, 'r')
columns = in_file.readline().strip('\n').split(',')
print('train.csv columns ', columns)
in_file_nm = 'test.csv'
in_file = open(in_file_nm, 'r')
columns = in_file.readline().strip('\n').split(',')
print('test.csv columns ', columns)

### 2.  load data to a Postgres db in docker container

In [ ]:
#view pg environment
try:
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")
    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [ ]:
#Create table sql queries
#""', '""', '""', '""
sent_movie_train = '''
CREATE TABLE sent_movie_train
(
 PhraseId	INTEGER PRIMARY KEY,
 SentenceId	INTEGER,
 Phrase	 TEXT,
 Sentiment INTEGER   
 )
'''
sent_movie_test = '''
CREATE TABLE sent_movie_test
(
 PhraseId	INTEGER PRIMARY KEY,
 SentenceId	INTEGER,
 Phrase	 TEXT
 )
'''

In [ ]:
#execute table table creations
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

try:
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    engine_string = "postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database)
    engine = create_engine(engine_string)
    
       
    cursor.execute(sent_movie_train)
    print('created table ', sent_movie_train)

    cursor.execute(sent_movie_test)
    print('created table ', sent_movie_test)

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while creating PostgreSQL table", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [ ]:
# good explaination of how top copy table https://www.laurivan.com/load-a-csv-file-with-header-in-postgres-via-psycopg/

SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

In [ ]:
#function to process files for created tables.
def process_file(conn, table_name, file_object):
    cursor = conn.cursor()
    cursor.copy_expert(sql=SQL_STATEMENT % table_name, file=file_object)
    conn.commit()
    cursor.close()

In [ ]:
#save processed csv files to pg db
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

try:
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    engine_string = "postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database)
    engine = create_engine(engine_string)
    
    file = open("train.csv")
    process_file(connection, 'sent_movie_train', file)
    print('populated table sent_movie_train')
    
    file = open("test.csv")
    process_file(connection, 'sent_movie_test', file)
    print('populated table sent_movie_test')   
    
except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while creating PostgreSQL table", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

### 3.  clean train and test tables to adapt Kaggle dataset to reflect more likely original data

#### clean train and test tables to show one sentence instead per review(sentiment)

In [ ]:
#create table of full sentences 
sql = '''
CREATE TABLE sent_train_tble AS
SELECT
s.phraseid,
s.sentenceid,
s.phrase,
s.sentiment
FROM(SELECT
  p.phraseid,
  p.sentenceid,
  p.phrase,
  p.sentiment,
  ROW_NUMBER() OVER(PARTITION BY p.sentenceid ORDER BY p.phraseid ASC) AS phraseid_rk
  FROM sent_movie_train p
  GROUP BY sentenceid, phraseid) s
where s.phraseid_rk = 1
'''

In [ ]:
try:
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    engine_string = "postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database)
    engine = create_engine(engine_string)
    
       
    cursor.execute(sql)
    print('created table sent_train_tble', sql)

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while creating PostgreSQL table", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [ ]:
#create table of full sentences 
sql = '''
CREATE TABLE sent_test_tble AS
SELECT
s.phraseid,
s.sentenceid,
s.phrase
FROM(SELECT
  p.phraseid,
  p.sentenceid,
  p.phrase,
  ROW_NUMBER() OVER(PARTITION BY p.sentenceid ORDER BY p.phraseid ASC) AS phraseid_rk
  FROM sent_movie_test p
  GROUP BY sentenceid, phraseid) s
where s.phraseid_rk = 1
'''

In [ ]:
try:
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    engine_string = "postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database)
    engine = create_engine(engine_string)
    
       
    cursor.execute(sql)
    print('created table sent_train_tble', sql)

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while creating PostgreSQL table", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")